In [1]:
import os
import sys
import numpy as np
import stingray
import scipy.io
from astropy.io import fits
import matplotlib.pyplot as plt
from nicer_fuc import *

/home/pl/anaconda3/lib/python3.11/site-packages/stingray/utils.py:35: UserWarning: pyfftw not installed. Using standard scipy fft
  warnings.warn("pyfftw not installed. Using standard scipy fft")


In [2]:
cwd = os.getcwd()
obs_list = get_obs(cwd)

In [3]:
for obs in obs_list:
    print_log(obs)
    obs = obs[0:10]

    lc_file = f'ni{obs}mpu7_sr1s.lc'
    lc_path = os.path.join(cwd,obs,lc_file)   

    with fits.open(lc_path) as data:

        #查看头文件信息
        header = data[1].header
        print(header)

        times = np.array(data[1].data['TIME'])
        rates = np.array(data[1].data['RATE'])

        src_name = data[1].header['object']
        print(src_name)

        ra,dec = get_ra_dec(src_name)
        print(ra,dec)

        timemjd = data[1].header['MJDREFI'] + data[1].header['MJDREFF'] + data[1].header['TSTART'] / 24 / 3600
        print(timemjd) #MJD时间

        t_zero = data[1].header['TIMEZERO']   #xspec能识别的时间戳
        print(t_zero)

        print()
        print(times[0])

        mjd_refi = data[1].header['MJDREFI']
        mjd_reff = data[1].header['MJDREFF']
        print(mjd_refi)

1050080126
XTENSION= 'BINTABLE'           / binary table extension                         BITPIX  =                    8 / 8-bit bytes                                    NAXIS   =                    2 / 2-dimensional binary table                     NAXIS1  =                   20 / width of table in bytes                        NAXIS2  =                 1739 / Number of bins                                 PCOUNT  =                    0 / size of special data area                      GCOUNT  =                    1 / one data group (required keyword)              TFIELDS =                    4 / number of fields in each row                   TTYPE1  = 'TIME    '           / label for field   1                            TFORM1  = 'D       '           / data format of field: 8-byte DOUBLE            TUNIT1  = 's       '           / physical unit of field                         TTYPE2  = 'RATE    '           / label for field   2                            TFORM2  = 'E       '         

In [4]:
from astropy.io import fits
import os

from astropy.io import fits
import os

def fits2dict(filename, par_list, ext):
    """
    从FITS文件的指定扩展中读取指定列或头文件键的数据并返回字典。

    参数:
    filename (str): FITS文件路径。
    par_list (list of str): 要提取的列名或头文件键名列表。
    ext (int): FITS文件的扩展号,默认将header全部添加。
        0 -> header, 如 'OBJECT', 'MJDREFI', 'MJDREFF', 'TIMEZERO' 等。
        1 -> EVENTS, 如 'TIME', 'RATE', 'ERROR' 等。
        2 -> FPM_SEL。
        3 -> GTI。
        4 -> PPS_TREND。

    返回:
    dict: 包含指定扩展数据的字典。
    """
    if type(filename) != str:
        raise TypeError("fits_file should be a string!")
    if type(ext) != int:
        raise TypeError("ext should be an integer!")
    if type(par_list) != list and type(par_list) != np.ndarray:
        raise TypeError("par_list should either be a list or an array!")

    with fits.open(filename) as hdul:
        results = {}

        header = hdul[0].header
        for key in par_list:
            if key.upper() in header:
                results[key] = header[key.upper()]

        data = hdul[ext].data
        if data is not None:
            for key in par_list:
                if key in data.names:
                    results[key] = data[key]
        else:
            raise ValueError(f"扩展 {ext} 不包含数据表。")

    return results

#光变示例
cwd = os.getcwd()
obs = '1050080126'
lc_file = f'ni{obs}mpu7_sr1s.lc'
lc_path = os.path.join(cwd, obs, lc_file)

par_list = ['TIME', 'RATE', 'ERROR', 'object', 'MJDREFI', 'MJDREFF', 'TIMEZERO']
#par_list = ['TIME','PI','PI_FAST','PI_RATIO']

data_dict = fits2dict(lc_path,par_list,1)
print(data_dict)

path_evt = '/home/pl/work/python-workspance/nicer-pl/1050080126/ni1050080126_0mpu7_cl.evt'
print(fits2dict(path_evt,['OBS_ID','START','STOP'],3))



{'object': '4U_1636-536', 'MJDREFI': 56658, 'MJDREFF': 0.000777592592592593, 'TIME': array([0.0000e+00, 1.0000e+00, 2.0000e+00, ..., 1.6925e+04, 1.6926e+04,
       1.6927e+04]), 'RATE': array([660., 650., 580., ..., 685., 658., 632.], dtype=float32), 'ERROR': array([25.690466, 25.495098, 24.083189, ..., 26.172504, 25.65151 ,
       25.13961 ], dtype=float32)}
{'OBS_ID': '1050080126', 'START': array([1.15028977e+08, 1.15029728e+08, 1.15030067e+08, 1.15045682e+08]), 'STOP': array([1.15029723e+08, 1.15030065e+08, 1.15030500e+08, 1.15045905e+08])}


In [5]:
att_var = ['TIME','MODE','SUBMODE_AZ','SUBMODE_EL']
mkf_var = ['TIME','ELV', 'BR_EARTH', 'SUNSHINE', 'FOV_FLAG', 'SUN_ANGLE',
           'MOON_ANGLE', 'ANG_DIST', 'SAA', 'SAA_TIME', 'COR_ASCA', 'COR_SAX',
           'MCILWAIN_L', 'NICER_SAA', 'TOT_ALL_COUNT', 'TOT_UNDER_COUNT',
           'TOT_OVER_COUNT', 'TOT_XRAY_COUNT']
hk_var = ['TIME','MPU_D_TEMP','MPU_A_TEMP','MPU_PWRBRDG_TEMP']


In [6]:
#att
att_path = '/home/pl/work/python-workspance/nicer-pl/ln/1050080128/auxil/ni1050080128.att.gz'
print(fits2dict(att_path,att_var,1))

{'TIME': array([1.18391359e+08, 1.18391359e+08, 1.18391359e+08, ...,
       1.18449173e+08, 1.18449173e+08, 1.18449173e+08]), 'MODE': array([1, 1, 1, ..., 1, 1, 1], dtype=uint8), 'SUBMODE_AZ': array([2, 2, 2, ..., 1, 1, 1], dtype=uint8), 'SUBMODE_EL': array([2, 2, 2, ..., 1, 1, 1], dtype=uint8)}


In [7]:
#cat
cat_path = '/home/pl/work/python-workspance/nicer-pl/ln/1050080128/auxil/ni1050080128.cat'
print(fits2dict(cat_path,['FILENAME','FORMAT'],1))


{'FILENAME': chararray(['xti/event_uf/ni1050080128_0mpu5_uf.evt',
           'xti/event_uf/ni1050080128_0mpu4_uf.evt',
           'xti/event_uf/ni1050080128_0mpu1_uf.evt',
           'xti/event_uf/ni1050080128_0mpu6_uf.evt',
           'xti/event_uf/ni1050080128_0mpu0_uf.evt',
           'xti/event_uf/ni1050080128_0mpu3_uf.evt',
           'xti/event_uf/ni1050080128_0mpu2_uf.evt',
           'xti/hk/ni1050080128_0mpu4.hk', 'xti/hk/ni1050080128_0mpu2.hk',
           'xti/hk/ni1050080128_0mpu5.hk', 'xti/hk/ni1050080128_0mpu0.hk',
           'xti/hk/ni1050080128_0mpu6.hk', 'xti/hk/ni1050080128_0mpu1.hk',
           'xti/hk/ni1050080128_0mpu3.hk',
           'xti/event_cl/ni1050080128_0mpu7_ufa.evt',
           'xti/event_cl/ni1050080128_0mpu7_cl.evt',
           'auxil/ni1050080128.orb', 'auxil/ni1050080128.cat',
           'auxil/ni1050080128.mkf', 'auxil/ni1050080128.att',
           'log/ni1050080128_joblog.html', 'log/ni1050080128_errlog.html'],
          dtype='<U64'), 'FORMAT': char

In [8]:
#mkf
mkf_path = '/home/pl/work/python-workspance/nicer-pl/ln/1050080128/auxil/ni1050080128.mkf'
print(fits2dict(mkf_path,['TIME','NICER_SAA','ANG_DIST'],1))

{'TIME': array([1.18391359e+08, 1.18391360e+08, 1.18391361e+08, ...,
       1.18449160e+08, 1.18449161e+08, 1.18449162e+08]), 'NICER_SAA': array([1, 1, 1, ..., 0, 0, 0], dtype=uint8), 'ANG_DIST': array([2.6391121e+01, 2.6389708e+01, 2.6123402e+01, ..., 6.9059203e-03,
       1.3806020e-03, 1.0438388e-02], dtype=float32)}


In [9]:
#orb
orb_path = '/home/pl/work/python-workspance/nicer-pl/ln/1050080128/auxil/ni1050080128.orb'
print(fits2dict(orb_path,['TIME','Vx','Vy'],1))


{'TIME': array([1.18391356e+08, 1.18391366e+08, 1.18391376e+08, ...,
       1.18449146e+08, 1.18449156e+08, 1.18449166e+08]), 'Vx': array([-6315.83973289, -6302.85145174, -6289.06145869, ...,
        5794.23187501,  5829.29467215,  5863.61012774]), 'Vy': array([-3752.69430032, -3810.05585701, -3866.93166503, ...,
        -250.74185907,  -180.8753436 ,  -110.98818823])}


In [10]:
#hk
print(fits2dict(path_evt,hk_var,1))


{'TIME': array([1.15028977e+08, 1.15028977e+08, 1.15028977e+08, ...,
       1.15045905e+08, 1.15045905e+08, 1.15045905e+08]), 'MPU_A_TEMP': array([34.53343915, 36.33558246, 35.91154874, ..., 37.28965833,
       34.95747287, 36.65360775])}


In [11]:
#uf_file = fits.open(cwd +'/ln/1050080126'+'/xti/event_uf/ni'+obsid+'_0mpu'+mpu_no+'_uf.evt')
#优化：将原始数据全部解压以保证文件名一致
def get_uf(obs,mpu_no):
    uf_file = f'{cwd}/ln/{obs}/xti/event_uf/ni{obs}_0mpu{mpu_no}_uf.evt.gz'
    uf_dic = fits2dict(uf_file,['TIME','RAWX','DEADTIME'],1)
    return uf_dic

print(get_uf(1050080126, 2))

{'TIME': array([1.15028722e+08, 1.15028722e+08, 1.15028722e+08, ...,
       1.15045909e+08, 1.15045909e+08, 1.15045909e+08]), 'RAWX': array([5, 5, 3, ..., 3, 5, 3], dtype=uint8), 'DEADTIME': array([2.26373e-05, 2.26373e-05, 1.51949e-05, ..., 1.51949e-05,
       2.26373e-05, 2.26373e-05])}


In [12]:

def get_ufa(obs,mpu_no):
    ufa_file = f'{cwd}/ln/{obs}/xti/event_cl/ni{obs}_0mpu{mpu_no}_ufa.evt'
    ufa_dic = fits2dict(ufa_file,['TIME','MPU_UNDER_COUNT','PI_RATIO'],1)
    return ufa_dic

print(get_ufa(1050080126, 7))

{'TIME': array([1.15028722e+08, 1.15028722e+08, 1.15028722e+08, ...,
       1.15045909e+08, 1.15045909e+08, 1.15045909e+08]), 'MPU_UNDER_COUNT': array([20, 19, 18, ..., 10,  6,  6], dtype=int32), 'PI_RATIO': array([       nan,        nan,        nan, ..., 0.93292683,        nan,
       0.9678899 ], dtype=float32)}


In [13]:
### RELEVANT PARAMETERS:

##### ATT file:
# Extension: ATTITUDE
# TIME, QPARAM, STATE, MODE, SUBMODE_AZ, SUBMODE_EL, ST_VALID, QUATSRC, FINEMEAS
# Extension: INST_ATTITUDE
# TIME, QPARAM, STATE, MODE, SUBMODE_AZ, SUBMODE_EL, ST_VALID, QUATSRC, FINEMEAS

##### CAT file:
# Extension: CATALOG
# FILENAME, FORMAT, TYPE, FILECLAS, DESCRIP, FILESIZE, ARCHSIZE, CHECKSUM, GZIP_CRC, CKSUM_B4

##### MKF file:
# Extension: ORIG_PREFILTER
# TIME, POSITION, VELOCITY, QUATERNION, PNTUNIT, POLAR, RA, DEC, ROLL, SAT_LAT,
# SAT_LON, SAT_ALT, ELV, BR_EARTH, SUNSHINE, FOV_FLAG, SUN_ANGLE, MOON_ANGLE,
# RAM_ANGLE, ANG_DIST, SAA, SAA_TIME, COR_ASCA, COR_SAX, MCILWAIN_L, SUN_RA, SUN_DEC,
# MOON_RA, MOON_DEC, EARTH_RA, EARTH_DEC, TIME_ADJ, ST_BBO, ST_VALID, ST_OBJECTS,
# ST_VIDEO_VDC, ATT_ANG_AZ, ATT_ANG_EL, RA_CMD, DEC_CMD, ATT_ERR_AZ, ATT_ERR_EL,
# ATT_STATE, ATT_MODE, ATT_SUBMODE_AZ, ATT_SUBMODE_EL, TARG_CMD, PPS_SOURCE,
# PPS_ERR_LOWPASS, GPS_INIT, GPS_CONVERGED, NICER_SAA, ST_STARS, ST_FAILCODE,
# MPU_ALL_COUNT, MPU_OVER_COUNT, MPU_UNDER_COUNT, MPU_XRAY_COUNT, TOT_ALL_COUNT,
# TOT_UNDER_COUNT, TOT_OVER_COUNT, TOT_XRAY_COUNT, FPM_ON, NUM_FPM_ON
# Extension: PREFILTER
# TIME, POSITION, VELOCITY, QUATERNION, PNTUNIT, POLAR, RA, DEC, ROLL, SAT_LAT,
# SAT_LON, SAT_ALT, ELV, BR_EARTH, SUNSHINE, FOV_FLAG, SUN_ANGLE, MOON_ANGLE,
# RAM_ANGLE, ANG_DIST, SAA, SAA_TIME, COR_ASCA, COR_SAX, MCILWAIN_L, SUN_RA, SUN_DEC,
# MOON_RA, MOON_DEC, EARTH_RA, EARTH_DEC, TIME_ADJ, ST_BBO, ST_VALID, ST_OBJECTS,
# ST_VIDEO_VDC, ATT_ANG_AZ, ATT_ANG_EL, RA_CMD, DEC_CMD, ATT_ERR_AZ, ATT_ERR_EL,
# ATT_STATE, ATT_MODE, ATT_SUBMODE_AZ, ATT_SUBMODE_EL, TARG_CMD, PPS_SOURCE,
# PPS_ERR_LOWPASS, GPS_INIT, GPS_CONVERGED, NICER_SAA, ST_STARS, ST_FAILCODE,
# MPU_ALL_COUNT, MPU_OVER_COUNT, MPU_UNDER_COUNT, MPU_XRAY_COUNT, TOT_ALL_COUNT,
# TOT_UNDER_COUNT, TOT_OVER_COUNT, TOT_XRAY_COUNT, FPM_ON, NUM_FPM_ON,
# FPM_RATIO_REJ_COUNT, FPM_XRAY_PI_0000_0025, FPM_XRAY_PI_0035_0200,
# FPM_XRAY_PI_0200_0800, FPM_XRAY_PI_0800_1200, FPM_XRAY_PI_1200_1500,
# FPM_XRAY_PI_1500_1700, FPM_XRAY_PI_COUNT, MPU_DEADTIME, MPU_DOUBLE_COUNT,
# MPU_FT_COUNT, MPU_NOISE25_COUNT, MPU_OVERONLY_COUNT, MPU_UNDERONLY_COUNT,
# FPM_DOUBLE_COUNT, FPM_OVERONLY_COUNT, FPM_UNDERONLY_COUNT, FPM_FT_COUNT,
# FPM_NOISE25_COUNT, MPU_FT_PI_AVG, MPU_FT_PI_ERR, MPU_FT_PI_FAST_AVG,
# MPU_FT_PI_FAST_ERR, MPU_NOISE25_PI_AVG, MPU_NOISE25_PI_ERR, ISS_ATT_STATE,
# ROBO_STATE, VEHICLE_SOYUZ_DC1, VEHICLE_SOYUZ_MRM1, VEHICLE_SOYUZ_MRM2,
# VEHICLE_SOYUZ_SM

##### ORB file:
# Extension: ORBIT
# TIME, X, Y, Z, Vx, Vy, Vz, GEONS_J2K_TIME_RAW0, GEONS_J2K_WEEK0, ORIG_TIME, QUALITY
# Extension: SPS_ORBIT
# TIME, X, Y, Z, Vx, Vy, Vz, TIME_VALID, QUALITY, SPS_SECS, SPS_WEEK, SDS_GDOP, ORIG_TIME

##### HK file:
# Extension: MPU_HK
#TIME, TICK_LOW32, MPU_P33D_VOLT, MPU_P5D_VOLT, MPU_M5D_VOLT, MPU_P33TEC_VOLT
# MPU_P33M_VOLT, MPU_GNDD_VOLT, MPU_HV_VOLT, MPU_D_TEMP, MPU_GNDA_VOLT,
# MPU_M5A_VOLT, MPU_P5A_VOLT, MPU_P25R_VOLT, MPU_P3R_VOLT, MPU_A_TEMP,
# MPU_PWRBRDG_TEMP, MPU_BAD_CSUM, MPU_INVALID_PKT, MPU_INVALID_ID, MPU_INVALID_MPU,
# MPU_LOWMEM_FIFO, MPU_LOWMEM,SCI, MPU_LOWMEM_OTHER, MPU_ALL_COUNT, MPU_OVER_COUNT,
# MPU_UNDER_COUNT, MPU_XRAY_COUNT, MPU_FPM_TEMP, MPU_PID_TEMP, MPU_HOTSIDE_TEMP,
# MPU_TEC_I, MPU_TEC_VOLT, MPU_BIAS_VOLT, MPU_FAST_LLD, MPU_SLOW_LLD, PACKET_FORMAT
# GIT_HASH, COMPILE_DATE

##### UF file:
# Extension: EVENTS
# TIME, RAWX, RAWY, PHA, PHA_FAST, DET_ID, DEADTIME, EVENT_FLAGS, TICK,
# Extension: GTI
# START, STOP
# Extension: PPS_TREND
# TIME, TICK, PKT_TICK

##### UFA file:
# Extension: EVENTS
# TIME, RAWX, RAWY, PHA, PHA_FAST, DET_ID, DEADTIME, EVENT_FLAGS, TICK, MPU_A_TEMP
# MPU_UNDER_COUNT, PI_FAST, PI, PI_RATIO
# Extension: GTI
# START, STOP


In [14]:
def evt2obs(eventfile):
    event = fits.open(eventfile)
    obs = event[0].header['OBS_ID']
    return obs
evt2obs(path_evt)

'1050080126'

In [24]:
from tqdm import tqdm
def psrpipe():
    pass

psrpipe_flags = ['--emin','0.3','--emax','12.0']

evt_list=[f'{cwd}/ln/{obs}/xti/event_cl/ni{obs}0mpu7_cl.evt' for obs in obs_list]

for i in tqdm(range(len(obs_list))):
    #psrpipe(eventfiles[i],['--emin','0.3','--emax','12.0','--nounderfilt'])
    cmd = f'psrpipe.py {cwd}/ln/{obs_list[i]} --emin 0.5 --emax 10 --nounderfilt'
    print(cmd)


100%|██████████| 2/2 [00:00<00:00, 8603.70it/s]

psrpipe.py /home/pl/work/python-workspance/nicer-pl/ln/0 --emin 0.5 --emax 10 --nounderfilt
psrpipe.py /home/pl/work/python-workspance/nicer-pl/ln/1 --emin 0.5 --emax 10 --nounderfilt
